## 模型构建流程：

* （一）数据简介：电力用户的95598工单数据、电量电费营销数据等为基础，综合分析电费敏感客户特征，建立客户电费敏感度模型，对电费敏感用户的敏感程度进行量化评判，帮助供电企业快速、准确的识别电费敏感客户，从而对应的提供有针对性的电费、电量提醒等精细化用电服务。


* 感谢大连理工大学团队的优胜方案

<img src="../img/3.png" style="width:1000px;height:480px;float:left">


* （二）模型架构：<img src="../img/2.png" style="width:1000px;height:580px;float:left">

基于敏感程度不同，构建二种模型！

<img src="../img/4.png" style="width:1000px;height:480px;float:left">

特征工程：

<img src="../img/11.png" style="width:800px;height:300px;float:left">


In [4]:
import pandas as pd
import numpy as np
import csv
import re
import os
import jieba
import pickle
from numpy import log
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

<img src="../img/5.png" style="width:1000px;height:680px;float:left">

选择3个主要信息表：

* 01工单信息 

* 02通话信息

* 09应收电费信息

In [5]:
data_path = '../rawdata/'
#95598 train 
file_jobinfo_train = '01_arc_s_95598_wkst_train.zip' 
# 95598 test
file_jobinfo_test = '01_arc_s_95598_wkst_test.zip'    
# 通话信息记录
file_comm = '02_s_comm_rec.zip'
# 应收电费信息表 train
file_flow_train = '09_arc_a_rcvbl_flow.zip'
# 应收电费信息表 test
file_flow_test = '09_arc_a_rcvbl_flow_test.zip'
# 训练集正例
file_label = 'train_label.zip'
# 测试集
file_test = 'test_to_predict.zip'

In [6]:
train_info = pd.read_csv(data_path + 'processed_' + file_jobinfo_train, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONE)
# 过滤CUST_NO为空的用户
train_info = train_info.loc[~train_info.CUST_NO.isnull()]
train_info['CUST_NO'] = train_info.CUST_NO.astype(np.int64)
train_info.head()

,APP_NO,ID,BUSI_TYPE_CODE,URBAN_RURAL_FLAG,ORG_NO,HANDLE_TIME,ACCEPT_CONTENT,HANDLE_OPINION,CALLING_NO,ELEC_TYPE,CUST_NO,PROV_ORG_NO,CITY_ORG_NO
0,2015101280016584,1000000527201688,1,2.0,3340501,2015/10/12 7:27:08,【一户无电】非居民客户报修一户无电，经指导客户检查，无法判断设备故障及其资产归属，请现场查处。,NaN,1625671.0,405.0,5115500151,33101,33405
1,2015101280016722,1000000527203612,3,2.0,3341020,2015/10/12 7:31:50,【银行代扣】客户咨询银行代扣事宜,NaN,1196922.0,202.0,5720017228,33101,33410
2,2015101280016734,1000000527203685,3,2.0,3340670,2015/10/12 7:32:03,【查询电费】客户查询本月电费金额，已告知。,NaN,1378624.0,201.0,6911007734,33101,33406
3,2015101280019989,1000000527220221,3,1.0,3340401,2015/10/12 8:04:42,【查询电费】客户查询本月电费金额。,NaN,1149071.0,202.0,4400013050,33101,33404
4,2015101280020073,1000000527220797,3,1.0,334010101,2015/10/12 8:06:05,【查询电费】客户查询电费是否欠费，已告知。,NaN,1052032.0,100.0,6023002461,33101,33401


重做数据索引，统计单数

In [7]:
train = train_info.CUST_NO.value_counts().to_frame().reset_index()
train.head()

,index,CUST_NO
0,6010619615,114
1,6010052424,83
2,5312672270,71
3,5513201131,71
4,3717006696,71


In [8]:
train.columns = ['CUST_NO', 'counts_of_jobinfo']
train.head()

,CUST_NO,counts_of_jobinfo
0,6010619615,114
1,6010052424,83
2,5312672270,71
3,5513201131,71
4,3717006696,71


加入label

In [9]:
temp = pd.read_csv(data_path + file_label, header=None)
temp.columns = ['CUST_NO']
train['label'] = 0
train.loc[train.CUST_NO.isin(temp.CUST_NO), 'label'] = 1
train = train[['CUST_NO', 'label', 'counts_of_jobinfo']]
train.head()

,CUST_NO,label,counts_of_jobinfo
0,6010619615,0,114
1,6010052424,1,83
2,5312672270,1,71
3,5513201131,1,71
4,3717006696,1,71


测试集做法相同，只是标签用-1来表示

In [10]:
test_info = pd.read_csv(data_path + 'processed_' + file_jobinfo_test, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONE)
test = test_info.CUST_NO.value_counts().to_frame().reset_index()
test.columns = ['CUST_NO', 'counts_of_jobinfo']
test['label'] = -1
test = test[['CUST_NO', 'label', 'counts_of_jobinfo']]
test.head()

,CUST_NO,label,counts_of_jobinfo
0,6010316261,-1,10
1,6010165238,-1,9
2,5312739722,-1,8
3,5001827156,-1,8
4,6010601157,-1,8


分两条路走，先来观察只有一条工单记录的

In [11]:
df = train.append(test).copy()
del temp, train, test

In [12]:
df = df.loc[df.counts_of_jobinfo == 1].copy()
df.reset_index(drop=True, inplace=True)
train = df.loc[df.label != -1]
test = df.loc[df.label == -1]
print('原始数据中的低敏感度用户分布情况如下：')
print('训练集：',train.shape[0])
print('正样本:',train.loc[train.label == 1].shape[0])
print('负样本:',train.loc[train.label == 0].shape[0])
print('-----------------------')
print('测试集：',test.shape[0])
df.drop(['counts_of_jobinfo'], axis=1, inplace=True)

原始数据中的低敏感度用户分布情况如下：
训练集： 401626
正样本: 13139
负样本: 388487
-----------------------
测试集： 327437


在总表中进行筛选，留下只有一条工单记录的数据

In [13]:
jobinfo = train_info.append(test_info).copy()
jobinfo = jobinfo.loc[jobinfo.CUST_NO.isin(df.CUST_NO)].copy()
jobinfo.reset_index(drop=True, inplace=True)
jobinfo = jobinfo.merge(df[['CUST_NO', 'label']], on='CUST_NO', how='left')
jobinfo.head()

,APP_NO,ID,BUSI_TYPE_CODE,URBAN_RURAL_FLAG,ORG_NO,HANDLE_TIME,ACCEPT_CONTENT,HANDLE_OPINION,CALLING_NO,ELEC_TYPE,CUST_NO,PROV_ORG_NO,CITY_ORG_NO,label
0,2015101280016722,1000000527203612,3,2.0,3341020,2015/10/12 7:31:50,【银行代扣】客户咨询银行代扣事宜,NaN,1196922.0,202.0,5720017228,33101,33410,0
1,2015101280031680,1000000527300870,3,2.0,334020106,2015/10/12 9:11:37,【咨询总户号】通过地址（户名）查询户号信息。,NaN,1073266.0,202.0,1658156188,33101,33402,0
2,2015101280032529,1000000527305820,3,2.0,334104001,2015/10/12 9:14:58,【咨询总户号】中介查户号,NaN,1058596.0,201.0,5812203341,33101,33410,0
3,2015101280017707,1000000527205167,3,2.0,334101001,2015/10/12 7:34:54,【停电信息】咨询计划停电信息，未查到,NaN,1581993.0,201.0,2311017103,33101,33410,0
4,2015101280020177,1000000527221507,3,1.0,3340201,2015/10/12 8:07:49,【停电信息】咨询计划停电信息,NaN,1468213.0,NaN,1610543109,33101,33402,0


加载通话数据

In [14]:
comm = pd.read_csv(data_path + file_comm, sep='\t')
print ('总数据量：',comm.shape)
comm.drop_duplicates(inplace=True)
print ('去掉重复无用的：',comm.shape)
comm.head()

总数据量： (1593088, 8)
去掉重复无用的： (1584351, 8)


,APP_NO,HANDLE_ID,COMM_NO,REQ_BEGIN_DATE,REQ_FINISH_DATE,ORG_NO,BUSI_TYPE_CODE,WKST_BUSI_TYPE_CODE
0,1000000619515828,NaN,1818693,2015/12/24 12:37:29,2015/12/24 12:39:39,13406,999.0,3
1,1000000267085485,NaN,1344134,2015/2/4 14:10:21,2015/2/4 14:11:49,15421,999.0,3
2,1000000241905876,NaN,1475375,2015/1/11 11:18:29,2015/1/11 11:23:20,1542316,999.0,15
3,1000000243214087,NaN,1634225,2015/1/12 16:50:25,2015/1/12 16:51:33,15404,3.0,3
4,1000000289386345,NaN,1316851,2015/3/3 21:51:13,2015/3/3 21:52:12,15404,999.0,3


过滤掉没出现在jobinfo中的数据

In [15]:
comm = comm.loc[comm.APP_NO.isin(jobinfo.ID)]
comm = comm.rename(columns={'APP_NO':'ID'})
comm = comm.merge(jobinfo[['ID', 'CUST_NO']], on='ID', how='left')
print ('可用数据量：',comm.shape)

可用数据量： (726248, 9)


过滤掉日期错误的

In [16]:
comm['REQ_BEGIN_DATE'] = comm.REQ_BEGIN_DATE.apply(lambda x:pd.to_datetime(x))
comm['REQ_FINISH_DATE'] = comm.REQ_FINISH_DATE.apply(lambda x:pd.to_datetime(x))

comm = comm.loc[~(comm.REQ_BEGIN_DATE > comm.REQ_FINISH_DATE)]
print ('过滤错误信息后数据量：',comm.shape) 
df = df.loc[df.CUST_NO.isin(comm.CUST_NO)].copy()

过滤错误信息后数据量： (726242, 9)


构建特征：通话时间，并进行归一化

In [17]:
comm['holding_time'] = comm['REQ_FINISH_DATE'] - comm['REQ_BEGIN_DATE']
comm['holding_time_seconds'] = comm.holding_time.apply(lambda x:x.seconds)
df = df.merge(comm[['CUST_NO', 'holding_time_seconds']], how='left', on='CUST_NO')
# 通话时间归一化，秒
df['holding_time_seconds'] = MinMaxScaler().fit_transform(df['holding_time_seconds'].values.reshape(-1, 1))
del comm
df.head()

,CUST_NO,label,holding_time_seconds
0,6025069204,0,0.027580
1,6024028361,0,0.019870
2,4611040160,0,0.043298
3,3951024135,0,0.063167
4,3951016564,0,0.025801


特征数据one-hot编码

In [18]:
jobinfo = jobinfo.loc[jobinfo.CUST_NO.isin(df.CUST_NO)].copy()
jobinfo.reset_index(drop=True, inplace=True)

rank函数会进行排名操作，也就是当前数据是排名第几的，指定max方法就是有相同的取大的名次。

In [19]:
df['rank_CUST_NO'] =df.CUST_NO.rank(method='max') 
df.head()

,CUST_NO,label,holding_time_seconds,rank_CUST_NO
0,6025069204,0,0.027580,537949.0
1,6024028361,0,0.019870,526429.0
2,4611040160,0,0.043298,286725.0
3,3951024135,0,0.063167,248166.0
4,3951016564,0,0.025801,247781.0


名词归一化

In [20]:
df['rank_CUST_NO'] = MinMaxScaler().fit_transform(df.rank_CUST_NO.values.reshape(-1, 1))
df.head()

,CUST_NO,label,holding_time_seconds,rank_CUST_NO
0,6025069204,0,0.027580,0.740729
1,6024028361,0,0.019870,0.724867
2,4611040160,0,0.043298,0.394806
3,3951024135,0,0.063167,0.341712
4,3951016564,0,0.025801,0.341182


对离散型数值进行编码

In [21]:
df = df.merge(jobinfo[['CUST_NO', 'BUSI_TYPE_CODE']], on='CUST_NO', how='left')
temp = pd.get_dummies(df.BUSI_TYPE_CODE, prefix='onehot_BUSI_TYPE_CODE', dummy_na=True)
df = pd.concat([df, temp], axis=1)
df.drop(['BUSI_TYPE_CODE'], axis=1, inplace=True)
del temp

In [22]:
df = df.merge(jobinfo[['CUST_NO', 'URBAN_RURAL_FLAG']], on='CUST_NO', how='left')
temp = pd.get_dummies(df.URBAN_RURAL_FLAG, prefix='onehot_URBAN_RURAL_FLAG', dummy_na=True)
df = pd.concat([df, temp], axis=1)
df.drop(['URBAN_RURAL_FLAG'], axis=1, inplace=True)
del temp

In [23]:
df.head()

,CUST_NO,label,holding_time_seconds,rank_CUST_NO,onehot_BUSI_TYPE_CODE_1.0,onehot_BUSI_TYPE_CODE_3.0,onehot_BUSI_TYPE_CODE_5.0,onehot_BUSI_TYPE_CODE_6.0,onehot_BUSI_TYPE_CODE_7.0,onehot_BUSI_TYPE_CODE_8.0,onehot_BUSI_TYPE_CODE_9.0,onehot_BUSI_TYPE_CODE_10.0,onehot_BUSI_TYPE_CODE_15.0,onehot_BUSI_TYPE_CODE_18.0,onehot_BUSI_TYPE_CODE_nan,onehot_URBAN_RURAL_FLAG_1.0,onehot_URBAN_RURAL_FLAG_2.0,onehot_URBAN_RURAL_FLAG_3.0,onehot_URBAN_RURAL_FLAG_nan
0,6025069204,0,0.027580,0.740729,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
1,6024028361,0,0.019870,0.724867,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
2,4611040160,0,0.043298,0.394806,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
3,3951024135,0,0.063167,0.341712,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
4,3951016564,0,0.025801,0.341182,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0


供电单位编码,按长度

In [24]:
df = df.merge(jobinfo[['CUST_NO', 'ORG_NO']], on='CUST_NO', how='left')
df['len_of_ORG_NO'] = df.ORG_NO.apply(lambda x:len(str(x)))
df.fillna(-1, inplace=True)

ration为数据集中正例子数据编码长度所占比例

In [25]:
train = df[df.label != -1] 
ratio = {}
for i in train.ORG_NO.unique():
    ratio[i] = len(train.loc[(train.ORG_NO == i) & (train.label == 1)]) / len(train.loc[train.ORG_NO == i])

In [26]:
df['ratio_ORG_NO'] = df.ORG_NO.map(ratio)
df['ratio_ORG_NO'].head()

0    0.042863
1    0.042863
2    0.000000
3    0.000000
4    0.000790
Name: ratio_ORG_NO, dtype: float64

同样进行one-hot编码

In [27]:
temp = pd.get_dummies(df.len_of_ORG_NO, prefix='onehot_len_of_ORG_NO')
df = pd.concat([df, temp], axis=1)
# drop
df.drop(['ORG_NO', 'len_of_ORG_NO'], axis=1, inplace=True)

时间数据处理:正负例之间有明显区别

<img src="../img/8.png" style="width:600px;height:380px;float:left">

In [28]:
df = df.merge(jobinfo[['CUST_NO', 'HANDLE_TIME']], on='CUST_NO', how='left')
df['date'] = df['HANDLE_TIME'].apply(lambda x:pd.to_datetime(x.split()[0]))
df['time'] = df['HANDLE_TIME'].apply(lambda x:x.split()[1])
# month
# 1.label encoder
df['month'] = df['date'].apply(lambda x:x.month)
# day 一个月的第几天
# 1.label encoder
df['day'] = df.date.apply(lambda x:x.day)
features = ['CUST_NO','date','time','month','day']
df[features].head()

,CUST_NO,date,time,month,day
0,6025069204,2015-03-10,10:54:06,3,10
1,6024028361,2015-05-11,9:43:33,5,11
2,4611040160,2015-11-25,14:43:23,11,25
3,3951024135,2015-04-10,15:20:03,4,10
4,3951016564,2015-08-03,16:27:40,8,3


按照上旬，中旬，下旬进行统计

In [29]:
df['is_in_first_tendays'] = 0
df.loc[df.day.isin(range(1,11)), 'is_in_first_tendays'] = 1
df['is_in_middle_tendays'] = 0
df.loc[df.day.isin(range(11,21)), 'is_in_middle_tendays'] = 1
df['is_in_last_tendays'] = 0
df.loc[df.day.isin(range(21,32)), 'is_in_last_tendays'] = 1

In [30]:
df['hour'] = df.time.apply(lambda x:int(x.split(':')[0]))
df.drop(['HANDLE_TIME', 'date', 'time'], axis=1, inplace=True)

同样的方法处理ELEC_TYPE，首位表示用电方式，如工业用电

In [31]:
df = df.merge(jobinfo[['CUST_NO', 'ELEC_TYPE']], on='CUST_NO', how='left')
df.fillna(0,inplace=True)
df['head_of_ELEC_TYPE'] = df.ELEC_TYPE.apply(lambda x:str(x)[0])

In [32]:
# 是否是空值
df['is_ELEC_TYPE_NaN'] = 0
df.loc[df.ELEC_TYPE == 0, 'is_ELEC_TYPE_NaN'] = 1
# 1.label encoder
df['label_encoder_ELEC_TYPE'] = LabelEncoder().fit_transform(df['ELEC_TYPE'])
# 2.ratio 
train = df[df.label != -1]
ratio = {}
for i in train.ELEC_TYPE.unique():
    ratio[i] = len(train.loc[(train.ELEC_TYPE == i) & (train.label == 1)]) / len(train.loc[train.ELEC_TYPE == i])
df['ratio_ELEC_TYPE'] = df.ELEC_TYPE.map(ratio)
df.fillna(0,inplace=True)

In [33]:
features = ['ratio_ELEC_TYPE','head_of_ELEC_TYPE']
df[features].head()

,ratio_ELEC_TYPE,head_of_ELEC_TYPE
0,0.045000,2
1,0.028253,0
2,0.033400,2
3,0.033400,2
4,0.033400,2


In [34]:
temp = pd.get_dummies(df.head_of_ELEC_TYPE, prefix='onehot_head_of_ELEC_TYPE')
df = pd.concat([df, temp], axis=1)
df.drop(['ELEC_TYPE', 'head_of_ELEC_TYPE'], axis=1, inplace=True)

城市编码，处理方式同上

In [35]:
df = df.merge(jobinfo[['CUST_NO', 'CITY_ORG_NO']], on='CUST_NO', how='left')
train = df[df.label != -1]
ratio = {}
for i in train.CITY_ORG_NO.unique():
    ratio[i] = len(train.loc[(train.CITY_ORG_NO == i) & (train.label == 1)]) / len(train.loc[train.CITY_ORG_NO == i])
df['ratio_CITY_ORG_NO'] = df.CITY_ORG_NO.map(ratio)
temp = pd.get_dummies(df.CITY_ORG_NO, prefix='onehot_CITY_ORG_NO')
df = pd.concat([df, temp], axis=1)
df.drop(['CITY_ORG_NO'], axis=1, inplace=True)

收费信息表数据

In [36]:
train_flow = pd.read_csv(data_path + file_flow_train, sep='\t')
test_flow = pd.read_csv(data_path + file_flow_test, sep='\t')
flow = train_flow.append(test_flow).copy()
flow.rename(columns={'CONS_NO':'CUST_NO'}, inplace=True)
flow.drop_duplicates(inplace=True)
flow = flow.loc[flow.CUST_NO.isin(df.CUST_NO)].copy()

In [37]:
flow.head()

,CUST_NO,RCVBL_YM,ORG_NO,PAY_MODE,T_PQ,RCVBL_AMT,RCVED_AMT,STATUS_CODE,RCVBL_PENALTY,RCVED_PENALTY,RISK_LEVEL_CODE,OWE_AMT,CONS_SORT_CODE,ELEC_TYPE_CODE,CTL_MODE
1,6010811670,201502,33401010130,10101.0,2791,2640.29,2640.29,1,0.00,0.00,NaN,2640.29,2.0,403.0,NaN
2,6010777305,201502,33401010130,20311.0,1883,1058.25,1058.25,1,2.12,2.12,NaN,1058.25,3.0,202.0,NaN
3,6010777296,201502,33401010130,20311.0,4279,3585.80,3585.80,1,0.00,0.00,NaN,3585.80,3.0,201.0,NaN
4,6010777190,201502,33401010130,20311.0,2514,1561.93,1561.93,1,0.00,0.00,NaN,1561.93,3.0,202.0,NaN
5,6010661530,201502,33401010130,20311.0,2574,2435.00,2435.00,1,0.00,0.00,NaN,2435.00,2.0,403.0,NaN


In [38]:
flow['T_PQ'] = flow.T_PQ.apply(lambda x:-x if x<0 else x)
flow['RCVBL_AMT'] = flow.RCVBL_AMT.apply(lambda x:-x if x<0 else x)
flow['RCVED_AMT'] = flow.RCVED_AMT.apply(lambda x:-x if x<0 else x)
flow['OWE_AMT'] = flow.OWE_AMT.apply(lambda x:-x if x<0 else x)

有些数据缺失了

In [39]:
df['has_biao9'] = 0
df.loc[df.CUST_NO.isin(flow.CUST_NO), 'has_biao9'] = 1

In [40]:
df['counts_of_09flow'] = df.CUST_NO.map(flow.groupby('CUST_NO').size())
df['counts_of_09flow'].head()

0    12.0
1    12.0
2     NaN
3     NaN
4     NaN
Name: counts_of_09flow, dtype: float64

构建统计特征

In [41]:
# 应收金额
df['sum_yingshoujine'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').RCVBL_AMT.sum()) + 1)
df['mean_yingshoujine'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').RCVBL_AMT.mean()) + 1)
df['max_yingshoujine'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').RCVBL_AMT.max()) + 1)
df['min_yingshoujine'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').RCVBL_AMT.min()) + 1)
df['std_yingshoujine'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').RCVBL_AMT.std()) + 1)
# 实收金额
df['sum_shishoujine'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').RCVED_AMT.sum()) + 1)
# 少交了多少
df['qianfei'] = df['sum_yingshoujine'] - df['sum_shishoujine']

# 总电量
df['sum_T_PQ'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').T_PQ.sum()) + 1)
df['mean_T_PQ'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').T_PQ.mean()) + 1)
df['max_T_PQ'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').T_PQ.max()) + 1)
df['min_T_PQ'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').T_PQ.min()) + 1)
df['std_T_PQ'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').T_PQ.std()) + 1)

# 电费金额
df['sum_OWE_AMT'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').OWE_AMT.sum()) + 1)
df['mean_OWE_AMT'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').OWE_AMT.mean()) + 1)
df['max_OWE_AMT'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').OWE_AMT.max()) + 1)
df['min_OWE_AMT'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').OWE_AMT.min()) + 1)
df['std_OWE_AMT'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').OWE_AMT.std()) + 1)

# 电费金额和应收金额差多少
df['dianfei_jian_yingshoujine'] = df['sum_OWE_AMT'] - df['sum_yingshoujine']

# 应收违约金
df['sum_RCVBL_PENALTY'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').RCVBL_PENALTY.sum()) + 1)
df['mean_RCVBL_PENALTY'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').RCVBL_PENALTY.mean()) + 1)
df['max_RCVBL_PENALTY'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').RCVBL_PENALTY.max()) + 1)
df['min_RCVBL_PENALTY'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').RCVBL_PENALTY.min()) + 1)
df['std_RCVBL_PENALTY'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').RCVBL_PENALTY.std()) + 1)

# 实收违约金
df['sum_RCVED_PENALTY'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').RCVED_PENALTY.sum()) + 1)
df['mean_RCVED_PENALTY'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').RCVED_PENALTY.mean()) + 1)
df['max_RCVED_PENALTY'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').RCVED_PENALTY.max()) + 1)
df['min_RCVED_PENALTY'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').RCVED_PENALTY.min()) + 1)
df['std_RCVED_PENALTY'] = log(df.CUST_NO.map(flow.groupby('CUST_NO').RCVED_PENALTY.std()) + 1)

df['chaduoshao_weiyuejin'] = df['sum_RCVBL_PENALTY'] - df['sum_RCVED_PENALTY']

In [42]:
# 每个用户有几个月的记录
df['nunique_RCVBL_YM'] = df.CUST_NO.map(flow.groupby('CUST_NO').RCVBL_YM.nunique())

# 平均每个月几条
df['mean_RCVBL_YM'] = df['counts_of_09flow'] / df['nunique_RCVBL_YM']
del train_flow, test_flow, flow

存下咱们的特征

In [43]:
if not os.path.isdir('../myfeatures'):
    os.makedirs('../myfeatures')
    
print('统计特征搞定！')
pickle.dump(df, open('../myfeatures/statistical_features_1.pkl', 'wb'))

统计特征搞定！


对文本特征：

* 1.增加专属词汇
* 2.分词
* 3.去停用词

<img src="../img/6.png" style="width:400px;height:130px;float:left">

In [44]:
print('开始处理表1中的文本特征...')
###############
# ACCEPT_CONTENT
###############

mywords = ['户号', '分时', '抄表', '抄表示数', '工单', '单号', '工单号', '空气开关', '脉冲灯', '计量表', '来电', '报修']
for word in mywords:
    jieba.add_word(word)

stops = set()
with open('../stopwords.txt', encoding='utf-8')as f:
    for word in f:
        word = word.strip()
        stops.add(word)

def fenci(line):
    res = []
    words = jieba.cut(line)
    for word in words:
        if word not in stops:
            res.append(word)
    return ' '.join(res)
print('分词ing...')
jobinfo['contents'] = jobinfo.ACCEPT_CONTENT.apply(lambda x:fenci(x))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Articuly\AppData\Local\Temp\jieba.cache


开始处理表1中的文本特征...


Loading model cost 0.794 seconds.
Prefix dict has been built succesfully.


分词ing...


手机号，户号等后面连接的号码是不用的，统一一下，具体号码没啥用

In [45]:
def hash_number(x):  
    shouji_pattern = re.compile('\s1\d{10}\s|\s1\d{10}\Z')
    if shouji_pattern.findall(x):
        x = re.sub(shouji_pattern, ' 手机number ', x)
    
    huhao_pattern = re.compile('\s\d{10}\s|\s\d{10}\Z')
    if huhao_pattern.findall(x):
        x = re.sub(huhao_pattern, ' 户号number ', x)
            
    tuiding_pattern = re.compile('\s\d{11}\s|\s\d{11}\Z')
    if tuiding_pattern.findall(x):
        x = re.sub(tuiding_pattern, ' 退订number ', x)
            
    gongdan_pattern = re.compile('\s201\d{13}\s|\s201\d{13}\Z')
    if gongdan_pattern.findall(x):
        x = re.sub(gongdan_pattern, ' 工单number ', x)
            
    tingdian_pattern = re.compile('\s\d{12}\s|\s\d{12}\Z')
    if tingdian_pattern.findall(x):
        x = re.sub(tingdian_pattern, ' 停电number ', x)
        
    return x.strip()

加入文本特征

In [46]:
jobinfo['content'] = jobinfo['contents'].apply(lambda x:hash_number(x))

jobinfo['len_of_contents'] = jobinfo.content.apply(lambda x:len(x.split()))
jobinfo['counts_of_words'] = jobinfo.content.apply(lambda x:len(set(x.split())))
text = df[['CUST_NO']].copy()
text = text.merge(jobinfo[['CUST_NO', 'len_of_contents', 'counts_of_words', 'content']], on='CUST_NO', how='left')
text = text.rename(columns={'content': 'contents'})

pickle.dump(text, open('../myfeatures/text_features_1.pkl', 'wb'))
print('done!')

done!


## 文本特征筛选

<img src="../img/9.png" style="width:800px;height:330px;float:left">

读取存下来的特征，构建数据集

In [47]:
import pickle
import pandas as pd
from scipy.sparse import csc_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb #https://www.lfd.uci.edu/~gohlke/pythonlibs/

print('select features...')

df = pickle.load(open('../myfeatures/statistical_features_1.pkl', 'rb'))
text = pickle.load(open('../myfeatures/text_features_1.pkl', 'rb'))
df = df.merge(text, on='CUST_NO', how='left')

train = df.loc[df.label != -1]
test = df.loc[df.label == -1]

x_data = train.copy()
x_val = test.copy()
# 打乱训练集
x_data = x_data.sample(frac=1, random_state=1).reset_index(drop=True)

select features...


稀疏矩阵

In [48]:
delete_columns = ['CUST_NO', 'label', 'contents']
X_train_1 = csc_matrix(x_data.drop(delete_columns, axis=1).values)         
X_val_1 = csc_matrix(x_val.drop(delete_columns, axis=1).values)

In [49]:
y_train = x_data.label.values
y_val = x_val.label.values

构造tf-idf特征

In [50]:
featurenames = list(x_data.drop(delete_columns, axis=1).columns)
print('creating tfidf...')
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=3, use_idf=False, smooth_idf=False, sublinear_tf=True)
tfidf.fit(x_data.contents)
word_names = tfidf.get_feature_names()
X_train_2 = tfidf.transform(x_data.contents)
print ('X_train_2[1]:',X_train_2[1])
X_val_2 = tfidf.transform(x_val.contents)
print('文本特征：{}维'.format(len(word_names)))
statistic_feature = featurenames.copy()
print('其他特征：{}维'.format(len(statistic_feature)))
featurenames.extend(word_names)
from scipy.sparse import hstack
X_train = hstack(((X_train_1), (X_train_2))).tocsc()
X_val = hstack(((X_val_1), (X_val_2))).tocsc()

print('特征数量',X_train.shape[1])
print('----------------------------------------------')

creating tfidf...
X_train_2[1]:   (0, 5829)	0.2902913877205925
  (0, 5888)	0.2902913877205925
  (0, 6430)	0.4915060446599552
  (0, 6436)	0.2902913877205925
  (0, 6438)	0.2902913877205925
  (0, 7948)	0.2902913877205925
  (0, 7963)	0.2902913877205925
  (0, 9786)	0.2902913877205925
  (0, 11147)	0.2902913877205925
  (0, 11151)	0.2902913877205925
文本特征：19596维
其他特征：85维
特征数量 19681
----------------------------------------------


基于特征选择来降维

In [51]:
print('采用xgboost筛选文本特征...')
print('training...')
dtrain = xgb.DMatrix(X_train, y_train, feature_names=featurenames)
dval = xgb.DMatrix(X_val, feature_names=featurenames)

params = {
    "objective": "binary:logistic",
    "booster": "gbtree",
    "eval_metric": "error",
    "eta": 0.1,
    'max_depth':12,
    'subsample':0.8,
    'min_child_weight':3,
    'colsample_bytree':1,
    'gamma':0.2,
    "lambda":300,
    "silent":1,
    'seed':1,
    'n_jobs':-1,
    'nthread':8,
}
watchlist = [(dtrain, 'train')]
model = xgb.train(params, dtrain, 2500, evals=watchlist, early_stopping_rounds=100, verbose_eval=100)

print('训练完毕。')
temp = pd.DataFrame.from_dict(model.get_fscore(), orient='index').reset_index()
temp.columns = ['feature', 'score']
temp.sort_values(['score'], axis=0, ascending=False, inplace=True)
temp.reset_index(drop=True, inplace=True)

print('留下文本特征数量：', len(temp.loc[~temp.feature.isin(statistic_feature)]))

selected_words = list(temp.loc[~temp.feature.isin(statistic_feature)].feature.values)
pickle.dump(selected_words, open('../myfeatures/single_select_words.pkl', 'wb'))

采用xgboost筛选文本特征...
training...
[18:58:59] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-error:0.00627
Will train until train-error hasn't improved in 100 rounds.
[100]	train-error:0.00603
[200]	train-error:0.00578
[300]	train-error:0.00559
[400]	train-error:0.00534
[500]	train-error:0.00516
[600]	train-error:0.00500
[700]	train-error:0.00481
[800]	train-error:0.00463
[900]	train-error:0.00446
[1000]	train-error:0.00421
[1100]	train-error:0.00401
[1200]	train-error:0.00384
[1300]	train-error:0.00365
[1400]	train-error:0.00347
[1500]	train-error:0.00332
[1600]	train-error:0.00314
[1700]	train-error:0.00298
[1800]	train-error:0.00284
[1900]	train-error:0.002

## 构建Xgboost模型

<img src="../img/10.png" style="width:1000px;height:500px;float:left">

读取特征

In [ ]:
import pickle
import pandas as pd
import numpy as np
import os
from scipy.sparse import csc_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb

def threshold(y, t):
    z = np.copy(y)
    z[z>=t] = 1
    z[z<t] = 0
    return z
print('training model...')
df = pickle.load(open('../myfeatures/statistical_features_1.pkl', 'rb'))
text = pickle.load(open('../myfeatures/text_features_1.pkl', 'rb'))
df = df.merge(text, on='CUST_NO', how='left')

train = df.loc[df.label != -1]
test = df.loc[df.label == -1]
print('训练集：',train.shape[0])
print('正样本:',train.loc[train.label == 1].shape[0])
print('负样本:',train.loc[train.label == 0].shape[0])
print('-----------------------')
print('测试集：',test.shape[0])
print('-----------------------')


基于选择的词来创建tf-idf，构建模型输入数据

In [ ]:
x_data = train.copy()
x_val = test.copy()
x_data = x_data.sample(frac=1, random_state=1).reset_index(drop=True)
#################
#################
###   input   ###
#################
#################
delete_columns = ['CUST_NO', 'label', 'contents']

X_train_1 = csc_matrix(x_data.drop(delete_columns, axis=1).values)
X_val_1 = csc_matrix(x_val.drop(delete_columns, axis=1).values)

y_train = x_data.label.values
y_val = x_val.label.values
featurenames = list(x_data.drop(delete_columns, axis=1).columns)
print('tfidf...')
select_words = pickle.load(open('../myfeatures/single_select_words.pkl', 'rb'))
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=3, use_idf=False, smooth_idf=False, sublinear_tf=True, vocabulary=select_words)
tfidf.fit(x_data.contents)
word_names = tfidf.get_feature_names()
X_train_2 = tfidf.transform(x_data.contents)
X_val_2 = tfidf.transform(x_val.contents)
print('文本特征：{}维'.format(len(word_names)))
statistic_feature = featurenames.copy()
print('其他特征：{}维'.format(len(statistic_feature)))
featurenames.extend(word_names)
from scipy.sparse import hstack
X_train = hstack(((X_train_1), (X_train_2))).tocsc()
X_val = hstack(((X_val_1), (X_val_2))).tocsc()

print('特征数量',X_train.shape[1])
print('----------------------------------------------')

bagging策略，不同随机方式3次来取最终结果

In [ ]:
print('start 3 xgboost!')
bagging = []
for i in range(1,4):
    print('group:',i)
    ##############
    #  xgboost
    ##############
    print('training...')
    dtrain = xgb.DMatrix(X_train, y_train, feature_names=featurenames)
    dval = xgb.DMatrix(X_val, feature_names=featurenames)

    params = {
        "objective": "binary:logistic",
        "booster": "gbtree",
        "eval_metric": "error",
        "eta": 0.1,
        'max_depth':14,
        'subsample':0.8,
        'min_child_weight':2,
        'colsample_bytree':1,
        'gamma':0.2,
        "lambda":300,
        'silent':1,
        "seed":i,
    }
    watchlist = [(dtrain, 'train')]

    model = xgb.train(params, dtrain, 2000, evals=watchlist,
                    early_stopping_rounds=50, verbose_eval=100)

    print('predicting...')
    y_prob = model.predict(dval, ntree_limit=model.best_ntree_limit)
    bagging.append(y_prob)
    print('---------------------------------------------')
print('gl hf !')

print('voting...')
t = 0.5
pres = []
for i in bagging:
    pres.append(threshold(i, t))
    
# vote
pres = np.array(pres).T.astype('int64')
result = []
for line in pres:
    result.append(np.bincount(line).argmax())
    
myout = test[['CUST_NO']].copy()
myout['pre'] = result
print('output!')
if not os.path.isdir('../result'):
    os.makedirs('../result')
myout.loc[myout.pre == 1, 'CUST_NO'].to_csv('../result/A.csv', index=False)